In [1]:
!python --version

Python 3.7.3


In [2]:
from tensortrade.data.cdd import CryptoDataDownload
import pandas as pd

#### Installing used libs

In [3]:
!pip install -U tensortrade==1.0.3 ta matplotlib tensorboardX scikit-learn

#### Checking if installed version of Tensortrade is the version 1.0.3

In [4]:
import tensortrade
print(tensortrade.__version__)

1.0.3


#### Importing used libs

In [5]:
import random

import ta
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import tensortrade.env.default as default
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.agents import A2CAgent
import tensortrade.stochastic as sp 
from tensortrade.oms.instruments import Instrument


#### Helper functions used to generate technical analysis features and create the environment for agent training and testing

In [6]:
def fetchTaFeatures(data):
    data = ta.add_all_ta_features(data, 'open', 'high', 'low', 'close', 'volume', fillna=True)
    data.columns = [name.lower() for name in data.columns]
    return data

def createEnv(config):

    coins = ["coin{}".format(x) for x in range(5)]
    bitfinex_streams = []

    with NameSpace("bitfinex"):
        for coin in coins:
            coinColumns = filter(lambda name: name.startswith(coin), config["data"].columns)
            bitfinex_streams += [
                Stream.source(list(config["data"][c]), dtype="float").rename(c) for c in coinColumns
            ]





    feed = DataFeed(bitfinex_streams)
    
    streams = []
    for coin in coins:
         streams.append(Stream.source(list(data[coin+":"+"close"]), dtype="float").rename("USD-"+coin))
    streams = tuple(streams)


    bitstamp = Exchange("bitfinex", service=execute_order)(
        Stream.source(list(data["coin0:close"]), dtype="float").rename("USD-BTC"),
        Stream.source(list(data["coin1:close"]), dtype="float").rename("USD-ETH"),
        Stream.source(list(data["coin2:close"]), dtype="float").rename("USD-TTC1"),
        Stream.source(list(data["coin3:close"]), dtype="float").rename("USD-TTC2"),
        Stream.source(list(data["coin4:close"]), dtype="float").rename("USD-TTC3"),
    
    )
    
    TTC1 = Instrument("TTC1", 8, "TensorTrade Coin1")
    TTC2 = Instrument("TTC2", 8, "TensorTrade Coin2")
    TTC3 = Instrument("TTC3", 8, "TensorTrade Coin3")
    
    
    cash = Wallet(bitstamp, 10000 * USD)
    asset = Wallet(bitstamp, 0 * BTC)
    asset1 = Wallet(bitstamp, 0 * ETH)
    asset2 = Wallet(bitstamp, 0 * TTC1)
    asset3 = Wallet(bitstamp, 0 * TTC2)
    asset4 = Wallet(bitstamp, 0 * TTC3)

    portfolio = Portfolio(USD, [cash, asset, asset1, asset2, asset3, asset4
        
    ])
    
    env = default.create(
      feed=feed,
      portfolio=portfolio,
      action_scheme="managed-risk", # mudei aqui e parcialmente funcionou
      reward_scheme='risk-adjusted', # substituir por sortino
      window_size=config["window_size"]
    )
    
    return env


#### Creating data with stochastic process for training

In [7]:
cdd = CryptoDataDownload()

coins = ["coin{}".format(x) for x in range(5)]
dfs = []
for coin in coins:
    df = sp.fbm(
    base_price=10000,
    base_volume=5000,
    start_date="2010-01-01",
    times_to_generate=100,
    time_frame='1H').add_prefix(coin+":")

    ta.add_all_ta_features(
    df,
    colprefix=coin+":",
    **{k: coin+":" + k for k in ['open', 'high', 'low', 'close', 'volume']}    )
    
    dfs.append(df)

data = pd.concat(dfs, axis=1)


/home/leandro/Documentos/ReinforcementLearningTrading/test/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/leandro/Documentos/ReinforcementLearningTrading/test/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/home/leandro/Documentos/ReinforcementLearningTrading/test/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/leandro/Documentos/ReinforcementLearningTrading/test/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
/home/leandro/Documentos/ReinforcementLearningTrading/test/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_

In [8]:
data

,coin0:open,coin0:high,coin0:low,coin0:close,coin0:volume,coin0:volume_adi,coin0:volume_obv,coin0:volume_cmf,coin0:volume_fi,coin0:volume_mfi,...,coin4:momentum_wr,coin4:momentum_ao,coin4:momentum_kama,coin4:momentum_roc,coin4:momentum_ppo,coin4:momentum_ppo_signal,coin4:momentum_ppo_hist,coin4:others_dr,coin4:others_dlr,coin4:others_cr
2010-01-01 00:00:00,10000.000000,10001.343151,9993.542037,10000.341722,300040.045281,2.230077e+05,3.000400e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.413598,NaN,0.000000
2010-01-01 01:00:00,10001.353324,10025.430291,10001.353324,10025.168824,299808.684120,5.163048e+05,5.998487e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.075026,-0.075054,-0.075026
2010-01-01 02:00:00,10026.378070,10029.459294,10019.737712,10024.889285,300553.492428,5.342845e+05,2.992952e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.147511,0.147403,0.072374
2010-01-01 03:00:00,10026.058313,10033.600988,10021.964946,10027.216152,299789.415482,5.050778e+05,5.990847e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.070139,-0.070163,0.002185
2010-01-01 04:00:00,10027.659622,10037.578759,10018.423317,10018.467698,299864.668954,2.066026e+05,2.992200e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.143716,0.143613,0.145904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-01-04 23:00:00,10066.655092,10076.354313,10063.318598,10070.047610,299820.215057,-1.882555e+06,-5.995351e+05,-0.043994,65644.354470,28.536184,...,-23.699890,-23.835610,10034.192301,-0.022327,-0.011181,-0.013152,0.001971,0.190135,0.189955,0.570699
2010-01-05 00:00:00,10070.198496,10070.520833,10057.151655,10057.151655,299657.009164,-2.182212e+06,-8.991921e+05,-0.109406,-495785.281139,28.550665,...,-31.291215,-21.832562,10034.312022,-0.005552,-0.006436,-0.011808,0.005372,-0.043536,-0.043546,0.526915
2010-01-05 01:00:00,10056.837729,10056.837729,10041.375210,10047.701845,300321.403160,-2.236774e+06,-1.199514e+06,-0.160492,-830384.574252,28.564320,...,-28.035099,-18.318119,10034.597447,0.000400,-0.004205,-0.010288,0.006083,0.018682,0.018680,0.545695
2010-01-05 02:00:00,10048.702368,10048.739120,10036.883688,10047.387848,299856.338476,-2.005273e+06,-1.499370e+06,-0.171965,-725208.797338,28.567477,...,-47.120707,-11.872555,10034.517959,0.008363,0.002242,-0.007782,0.010023,-0.146038,-0.146144,0.398860


#### Adding technical analysis features to the dataframe

#### Normalizing the data

In [9]:
scaler = MinMaxScaler()
# delColumns = ["coin{}:date".format(x) for x in range(5)]
# delColumns += ["coin{}:unix".format(x) for x in range(5)]

# data = data.drop(columns=delColumns)
norm_data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

In [10]:
data

,coin0:open,coin0:high,coin0:low,coin0:close,coin0:volume,coin0:volume_adi,coin0:volume_obv,coin0:volume_cmf,coin0:volume_fi,coin0:volume_mfi,...,coin4:momentum_wr,coin4:momentum_ao,coin4:momentum_kama,coin4:momentum_roc,coin4:momentum_ppo,coin4:momentum_ppo_signal,coin4:momentum_ppo_hist,coin4:others_dr,coin4:others_dlr,coin4:others_cr
2010-01-01 00:00:00,10000.000000,10001.343151,9993.542037,10000.341722,300040.045281,2.230077e+05,3.000400e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.413598,NaN,0.000000
2010-01-01 01:00:00,10001.353324,10025.430291,10001.353324,10025.168824,299808.684120,5.163048e+05,5.998487e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.075026,-0.075054,-0.075026
2010-01-01 02:00:00,10026.378070,10029.459294,10019.737712,10024.889285,300553.492428,5.342845e+05,2.992952e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.147511,0.147403,0.072374
2010-01-01 03:00:00,10026.058313,10033.600988,10021.964946,10027.216152,299789.415482,5.050778e+05,5.990847e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.070139,-0.070163,0.002185
2010-01-01 04:00:00,10027.659622,10037.578759,10018.423317,10018.467698,299864.668954,2.066026e+05,2.992200e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.143716,0.143613,0.145904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-01-04 23:00:00,10066.655092,10076.354313,10063.318598,10070.047610,299820.215057,-1.882555e+06,-5.995351e+05,-0.043994,65644.354470,28.536184,...,-23.699890,-23.835610,10034.192301,-0.022327,-0.011181,-0.013152,0.001971,0.190135,0.189955,0.570699
2010-01-05 00:00:00,10070.198496,10070.520833,10057.151655,10057.151655,299657.009164,-2.182212e+06,-8.991921e+05,-0.109406,-495785.281139,28.550665,...,-31.291215,-21.832562,10034.312022,-0.005552,-0.006436,-0.011808,0.005372,-0.043536,-0.043546,0.526915
2010-01-05 01:00:00,10056.837729,10056.837729,10041.375210,10047.701845,300321.403160,-2.236774e+06,-1.199514e+06,-0.160492,-830384.574252,28.564320,...,-28.035099,-18.318119,10034.597447,0.000400,-0.004205,-0.010288,0.006083,0.018682,0.018680,0.545695
2010-01-05 02:00:00,10048.702368,10048.739120,10036.883688,10047.387848,299856.338476,-2.005273e+06,-1.499370e+06,-0.171965,-725208.797338,28.567477,...,-47.120707,-11.872555,10034.517959,0.008363,0.002242,-0.007782,0.010023,-0.146038,-0.146144,0.398860


In [11]:
norm_data

,coin0:open,coin0:high,coin0:low,coin0:close,coin0:volume,coin0:volume_adi,coin0:volume_obv,coin0:volume_cmf,coin0:volume_fi,coin0:volume_mfi,...,coin4:momentum_wr,coin4:momentum_ao,coin4:momentum_kama,coin4:momentum_roc,coin4:momentum_ppo,coin4:momentum_ppo_signal,coin4:momentum_ppo_hist,coin4:others_dr,coin4:others_dlr,coin4:others_cr
0,0.496862,0.491473,0.495223,0.499483,0.533533,0.895431,0.666418,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.242710
1,0.502620,0.594519,0.528470,0.604479,0.425844,0.993960,0.749609,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.482865,0.299417,0.190391
2,0.609095,0.611755,0.606717,0.603297,0.772519,1.000000,0.666211,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.800244,0.729683,0.293179
3,0.607735,0.629473,0.616197,0.613137,0.416876,0.990188,0.749397,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.489835,0.308877,0.244233
4,0.614548,0.646490,0.601123,0.576139,0.451903,0.889920,0.666190,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.794831,0.722353,0.344454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.780465,0.812373,0.792207,0.794275,0.431212,0.188097,0.416801,0.527214,0.281905,0.221361,...,0.763001,0.290120,0.711192,0.591797,0.285572,0.020428,0.470608,0.861033,0.811986,0.640679
96,0.795542,0.787417,0.765959,0.739737,0.355247,0.087432,0.333652,0.399991,0.176183,0.221586,...,0.687088,0.311255,0.712448,0.601342,0.385089,0.076109,0.554741,0.527775,0.360359,0.610147
97,0.738695,0.728880,0.698811,0.699773,0.664492,0.069102,0.250318,0.300631,0.113176,0.221798,...,0.719649,0.348337,0.715442,0.604728,0.431883,0.139152,0.572308,0.616510,0.480714,0.623243
98,0.704080,0.694234,0.679694,0.698445,0.448025,0.146872,0.167113,0.278318,0.132981,0.221847,...,0.528793,0.416347,0.714608,0.609259,0.567103,0.243038,0.669782,0.381589,0.161918,0.520850


#### Creating a training environment with a 10-period window

In [12]:
config = {
            "window_size": 10,
            "data": norm_data
        }

env = createEnv(config)

#### Running agent training with 100,000 steps and 10 episodes.
The agent will be saved in the "agents/" folder

In [13]:
!mkdir -p agents/

agent = A2CAgent(env)
reward = agent.train(n_steps=20000, save_path="agents/", n_episodes = 10)

====      AGENT ID: 13fc7a6b-aa96-4bd0-b52c-3da6df8c4e6a      ====
====      EPISODE ID (1/10): dd7d3634-ffc9-47e9-8fec-269f486398a5      ====
====      EPISODE ID (2/10): 43bd2e6f-0d73-4816-ba52-f869bcb317f6      ====


/home/leandro/Documentos/ReinforcementLearningTrading/test/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


====      EPISODE ID (3/10): 5636fe6d-dc29-4d25-944f-45c0fb12c046      ====
====      EPISODE ID (4/10): 08fa3bc8-c9fa-4452-9460-b4a7a62a5ca6      ====
====      EPISODE ID (5/10): 3adbf1d7-b806-4c2b-adb5-140e33339945      ====
====      EPISODE ID (6/10): 78d6bf0b-7b23-46be-b49a-bf626d84c23f      ====


KeyboardInterrupt: 

#### Creating a new environment to check agent performance in training

In [ ]:
env = createEnv({
    "window_size": 10, 
    "data": norm_data
})


#### Require action until step returns Done == True

In [ ]:
episode_reward = 0
done = False
obs = env.reset()

while not done:
    action = agent.get_action(obs)
    obs, reward, done, info = env.step(action)
    episode_reward += reward

In [ ]:
reward

#### Plotting agent performance

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

fig.suptitle("Performance")

axs[0].plot(np.arange(len(data["BTC:close"])), data["BTC:close"], label="price")
axs[0].set_title("Trading Chart")

performance_df = pd.DataFrame().from_dict(env.action_scheme.portfolio.performance, orient='index')
performance_df.plot(ax=axs[1])
axs[1].set_title("Net Worth")

plt.show()

#### Create new data to assess how the agent behaves in data you've never seen

In [ ]:
data2 = sp.fbm(
    base_price=10000,
    base_volume=5000,
    start_date="2010-01-01",
    times_to_generate=10000,
    time_frame='1H'
)
data2 = data2.reset_index(drop=True)
data2.close.plot()


In [ ]:
data2 = fetchTaFeatures(data2)

In [ ]:
norm_data = pd.DataFrame(scaler.transform(data2), columns=data2.columns)

In [ ]:
env = createEnv({
    "window_size": 10, 
    "data": norm_data
})

episode_reward = 0
done = False
obs = env.reset()

while not done:
    action = agent.get_action(obs)
    obs, reward, done, info = env.step(action)
    episode_reward += reward

#### Ploting performance in test data

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

fig.suptitle("Performance")

axs[0].plot(np.arange(len(data2["close"])), data2["close"], label="price")
axs[0].set_title("Trading Chart")

performance_df = pd.DataFrame().from_dict(env.action_scheme.portfolio.performance, orient='index')
performance_df.plot(ax=axs[1])
axs[1].set_title("Net Worth")
# axs[1].set_xlim(0, 800)

plt.show()